# Part I. ETL Pipeline for Pre-Processing the Files

## PRE-PROCESSING THE FILES

#### Importing Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:

print(os.getcwd())

filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables and answering analytical queries on the DB

#### Steps:
- Initiate an empty list of rows that will be generated from each file
- For every filepath in the file path list read csv file,creating a csv reader objectthe extract each data row one by one and append it.
- Create a smaller csv for use in Cassendra DB

In [3]:

full_data_rows_list = [] 
    

for f in file_path_list:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)       
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# Checking the total number of rows 
print(len(full_data_rows_list))
# checking how the list of event data rows will looks 
print(full_data_rows_list[:5])

# creating a smaller event data csv file called that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
[['Stephen Lynch', 'Logged In', 'Jayden', 'M', '0', 'Bell', '182.85669', 'free', 'Dallas-Fort Worth-Arlington, TX', 'PUT', 'NextSong', '1.54099E+12', '829', "Jim Henson's Dead", '200', '1.54354E+12', '91'], ['Manowar', 'Logged In', 'Jacob', 'M', '0', 'Klein', '247.562', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '1049', 'Shell Shock', '200', '1.54354E+12', '73'], ['Morcheeba', 'Logged In', 'Jacob', 'M', '1', 'Klein', '257.41016', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '1049', 'Women Lose Weight (Feat: Slick Rick)', '200', '1.54354E+12', '73'], ['Maroon 5', 'Logged In', 'Jacob', 'M', '2', 'Klein', '231.23546', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '1049', "Won't Go Home Without You", '200', '1.54354E+12', '73'], ['Train', 'Logged In', 'Jacob', 'M', '3', 'Klein', '216.76363', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '1049'

#### checking the number of rows in the csv file

In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster to make a connection to a Cassandra instance on local machine (127.0.0.1)

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Creating Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set KEYSPACE to the keyspace created above

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### Understanding the overview of the data.

In [8]:
pd.read_csv('event_datafile_new.csv').head(5)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Stephen Lynch,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",829,Jim Henson's Dead,91
1,Manowar,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Shell Shock,73
2,Morcheeba,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Women Lose Weight (Feat: Slick Rick),73
3,Maroon 5,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Won't Go Home Without You,73
4,Train,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Hey_ Soul Sister,73


In [9]:
temp=pd.read_csv('event_datafile_new.csv')

In [10]:
temp.columns

Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')

## Problem Scenaio:
## Create queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and         itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


## We will visit each scenarios and Understand how to model.

#### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
#### Steps: 
- First to model the table based on fact that  we need to identity the artists , songs and its length based on the sessionID and ItemInsession we need our data to be partitioned and sorted accordingly , so we choose our composite key containg them both in order to choose our data uniquely and efficiently.
- Next we will be reading the data from `event_datafile_new.csv` and INSERT then in the table created accordingly.
- Now that our data is interted we will query the table for artist ,song title and song length based on the sessionid and itemInsession above and verify.

In [11]:
query=" DROP table if exists music_history"
try:
    session.execute(query)

except Exception as e:
    print(e)

query="create table if not exists music_history"
query=query + "(sessionId int,itemInSession int, artist text,song text, length float ,PRIMARY KEY(sessionId,itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO music_history (sessionId,itemInSession,artist,song,length)"
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]),line[0],line[9],float(line[5])))

#### Querying the table to verify that the data is as expected

In [13]:
query="select artist,song,length from music_history where sessionId = 338 and itemInSession = 4"
try:
    rows=session.execute(query)
except Exception as e:
    print(e)

for item in rows:
    print(item.artist,item.song,item.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)for userid = 10,  sessionid = 182

#### Steps:
- Here the challenge is modelling a table for the query so that we can fetch by `userid and sessionId` sorted by `itemInSession`. Now as we already know that primary key partitions the data and clustering key sorts it then, The best way to model this query is if we choose partition as `userid and sessionId` and clustering key as `itemInSession` as a part of the composite key. ie,`(user_id,sessionId),itemInSession)`.
- Next we will be reading the data from `event_datafile_new.csv` and INSERT then in the table created accordingly.
- Finally we select artist,song,first_name,last_name from user_history where user_id = 10 and sessionid = 182 


In [14]:
query=" DROP table if exists user_history"
try:
    session.execute(query)

except Exception as e:
    print(e)

query="create table if not exists user_history"
query=query + "(user_id int,sessionId int,itemInSession int,artist text,song text,first_name text,last_name text,PRIMARY KEY((user_id,sessionId),itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_history (user_id,sessionId,itemInSession,artist,song,first_name,last_name)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))

#### Querying the table to verify that the data is as expected

In [16]:
query="select artist,song,first_name,last_name from user_history where user_id = 10 and sessionid = 182 "
try:
    rows=session.execute(query)
except Exception as e:
    print(e)

for item in rows:
    print(item.artist,'\t',item.song,'\t',item.first_name,item.last_name)

Down To The Bone 	 Keep On Keepin' On 	 Sylvie Cruz
Three Drives 	 Greece 2000 	 Sylvie Cruz
Sebastien Tellier 	 Kilometer 	 Sylvie Cruz
Lonnie Gordon 	 Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 	 Sylvie Cruz


 ## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
 #### Steps:
 - We will first create a table based on the fact that we want our results based on the people who listened to a particular song , so we    choose our primary key to be song , Now to make it unique we take user_id as the combination so here we have a composite key consisting of the partition and clustering key respectively.
 
 - we load the data from `event_datafile_new.csv` and write the INSERT accordingly.
 - Next we select the first name and last name from our table where the song is the `All Hands Against His Own`

In [17]:

query=" DROP table if exists music_app_history"
try:
    session.execute(query)

except Exception as e:
    print(e)

query="create table if not exists music_app_history"
query=query + "(song text,user_id int,first_name text,last_name text,PRIMARY KEY(song,user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO music_app_history (song,user_id,first_name,last_name)"
        query = query + "VALUES(%s,%s,%s,%s)"
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

#### Querying the table to verify that the data is as expected

In [19]:
query="select first_name,last_name from music_app_history where song = 'All Hands Against His Own' "
try:
    rows=session.execute(query)
except Exception as e:
    print(e)

for item in rows:
    print(item.first_name,'\t',item.last_name)

Jacqueline 	 Lynch
Tegan 	 Levine
Sara 	 Johnson


### Dropping each of the tables before closing out the sessions

In [20]:
## TO-DO: Drop the table before closing out the sessions
query= " DROP table if exists music_history"
try:
    session.execute(query)

except Exception as e:
    print(e)
    

In [21]:
query= " DROP table if exists user_history"
try:
    session.execute(query)

except Exception as e:
    print(e)

In [22]:
query= " DROP table if exists music_app_history"
try:
    session.execute(query)

except Exception as e:
    print(e)

### Closing the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()